In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import re  
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
plt.rcParams['figure.figsize'] = [8,5]
plt.rcParams['font.size'] =14
plt.rcParams['font.weight']= 'bold'
sns.set()

In [2]:
import torchvision
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets 
import torchvision.transforms as transforms

In [3]:
VGG16 = [64,64,'M',128,128,'M',256,256,256,'M',512,512,512,'M',512,512,512,'M']

In [14]:
class VGG_Net(nn.Module):
    def __init__(self , in_channels =3 , num_classes= 1000):
        super().__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG16)
        self.fc_layers = nn.Sequential(
        nn.Linear(512*7*7 , 4096),
        nn.ReLU(),
        nn.Dropout(p = 0.5),
        nn.Linear(4096 , 4096),
        nn.ReLU(),
        nn.Dropout(p = 0.5),
        nn.Linear(4096 , num_classes)
        )
    def forward(self , x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0] , -1)
        return self.fc_layers(x)
    
    def create_conv_layers(self , arch):
        layers = []
        in_channels = self.in_channels
        for x in arch:
            if type(x) == int:
                layers += [nn.Conv2d(in_channels=in_channels , out_channels=x ,
                                    kernel_size=3 , padding=1 , stride = 1),
                          nn.BatchNorm2d(x),
                          nn.ReLU()
                          ]
                in_channels = x
            else:
                layers +=[nn.MaxPool2d(kernel_size=2)]
        return nn.Sequential(*layers)

In [15]:
model = VGG_Net(in_channels=3, num_classes = 1000).to('cuda')
model

VGG_Net(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [16]:
print(model(torch.randn(1,3,224,224).to('cuda')).shape)

torch.Size([1, 1000])
